### Extract First Name and Last Name from Full Name

Given a PySpark DataFrame containing a column `name` with **full names** (e.g., `"Rohish Zade"`), you are required to: create two new columns: `first_name` and `last_name`.

#### Sample Data

In [0]:
data = [ 
    (1, "Rohish Zade", 2000),
    (2, "Priya Ramteke", 2000),
    (3, "Faizal Reza", 2000),
]
columns = ["id", "full_name", "salary"]

df = spark.createDataFrame(data, columns)
df.show()

+---+-------------+------+
| id|    full_name|salary|
+---+-------------+------+
|  1|  Rohish Zade|  2000|
|  2|Priya Ramteke|  2000|
|  3|  Faizal Reza|  2000|
+---+-------------+------+



#### Solution 1:
Using `split()` Function

In [0]:
from pyspark.sql.functions import col, split, substring, instr, expr

In [0]:
split_df = df.withColumn("first_name", split(col("full_name"), " ").getItem(0)) \
            .withColumn("last_name", split(col("full_name"), " ").getItem(1))

split_df.show()

+---+-------------+------+----------+---------+
| id|    full_name|salary|first_name|last_name|
+---+-------------+------+----------+---------+
|  1|  Rohish Zade|  2000|    Rohish|     Zade|
|  2|Priya Ramteke|  2000|     Priya|  Ramteke|
|  3|  Faizal Reza|  2000|    Faizal|     Reza|
+---+-------------+------+----------+---------+



**Explanation:**
- `split(col("name"), " ")` splits the string at the space into an array like `["Rohish", "Zade"]`.
- `.getItem(0)` fetches the first element (first name).
- `.getItem(1)` fetches the second element (last name).

### Solution 2:

Using `substring()` and `instr()` (Extract Substrings Based on Position)

**What is instr() in PySpark?**
- `instr()` is a string function in PySpark that returns the 1-based position of the first occurrence of a substring in a string column. If the substring is not found, it returns 0.
- Syntax: `instr(column, substring)`
- Indexing is 1-based, not 0-based like Python

In [0]:
# Example:
df.withColumn("space_index", instr(col("full_name"), " ")).show()

+---+-------------+------+-----------+
| id|    full_name|salary|space_index|
+---+-------------+------+-----------+
|  1|  Rohish Zade|  2000|          7|
|  2|Priya Ramteke|  2000|          6|
|  3|  Faizal Reza|  2000|          7|
+---+-------------+------+-----------+



In [0]:
df.withColumn("space_index", instr(col("full_name"), " ")).show()

+---+-------------+------+-----------+
| id|    full_name|salary|space_index|
+---+-------------+------+-----------+
|  1|  Rohish Zade|  2000|          7|
|  2|Priya Ramteke|  2000|          6|
|  3|  Faizal Reza|  2000|          7|
+---+-------------+------+-----------+



In [0]:
substr_df = df.withColumn("space_index", instr(col("full_name"), " ")) \
               .withColumn("first_name", expr("substring(full_name, 1, space_index - 1)")) \
               .withColumn("last_name", expr("substring(full_name, space_index + 1, len(full_name))")).drop("space_index")

substr_df.show()

+---+-------------+------+----------+---------+
| id|    full_name|salary|first_name|last_name|
+---+-------------+------+----------+---------+
|  1|  Rohish Zade|  2000|    Rohish|     Zade|
|  2|Priya Ramteke|  2000|     Priya|  Ramteke|
|  3|  Faizal Reza|  2000|    Faizal|     Reza|
+---+-------------+------+----------+---------+



**Explanation:**
- `instr(full_name, " ")` finds the position of space.
- `substring(full_name, 1, space_index - 1)` gives the first name.
- `substring(full_name, space_index + 1, length(full_name))` gets the rest of the name (last name).